# The theory

This is calculated as per section 3.3 in Lister et al. (2011). The $\gamma$-ray loudness, $G$, is defined as

$$ G = \frac{L_\gamma}{L_\mathrm{r}} $$

where $L_\gamma$ is the $\gamma$-ray luminosity and $L_\mathrm{r}$ is the radio luminosity. We can calculate $L_\gamma$ according to

$$ L_\gamma = \frac{4 \, \pi \, d^2 \, S_\gamma}{(1 + z)^{2 - \Gamma}} $$

where $d$ is the luminosity distance, $S_\gamma$ is the energy flux ($\mathrm{erg}\,\mathrm{cm}^{-2}\,\mathrm{s}^{-1}$), $z$ is the redshift, and $\Gamma$ is the photon spectral index. We can find $L_\mathrm{r}$ using

$$ L_\mathrm{r} = \frac{4 \, \pi \, d^2 \, \Delta \nu \, S_\mathrm{r}}{(1 + z)} $$

where $\Delta\nu$ is the bandwidth of the radio measurement, $S_\mathrm{r}$ is the radio flux density ($\mathrm{Jy}$), and we have assumed a flat radio spectral index. Then we have

$$ G = \frac{S_\gamma}{\Delta \nu \, S_\mathrm{r}} \, \frac{1}{(1 + z)^{1 - \Gamma}} $$

The uncertainty on $G$ is

\begin{align*}
\Delta G &= \sqrt{\bigg({\frac{\delta G}{\delta S_\gamma}} \, \Delta S_\gamma \bigg)^2 + \bigg({\frac{\delta G}{\delta S_\mathrm{r}}} \, \Delta S_\mathrm{r} \bigg)^2 + \bigg({\frac{\delta G}{\delta \Gamma}} \, \Delta \Gamma \bigg)^2} \\
&= \sqrt{ \bigg( \frac{1}{\Delta \nu \, S_\mathrm{r}} \, \frac{1}{(1 + z)^{1 - \Gamma}} \, \Delta S_\gamma\bigg)^2 +
\bigg(\frac{-S_\gamma}{\Delta \nu \, S_\mathrm{r}^2} \, \frac{1}{(1 + z)^{1 - \Gamma}} \, \Delta S_\mathrm{r}\bigg)^2 +
\bigg( \frac{S_\gamma}{\Delta \nu \, S_\mathrm{r}} \, \frac{\ln (z + 1)}{(1 + z)^{1 - \Gamma}} \, \Delta \Gamma \bigg)^2}
\end{align*}

# Get the data

In [1]:
from astropy.io import fits
from numpy      import corrcoef, log, mean, sqrt
from ipynb.fs.full.redshift import get_data

In [2]:
filename      = 'gamma-ray-loudness.txt'
sources       = ['3FGL J1058.6+5627', '3FGL J1117.9+5355', '3FGL J1124.9+4932', '3FGL J1138.2+4905', '3FGL J1153.4+4932', '3FGL J1208.7+5442', '3FGL J1212.6+5135', '3FGL J1215.0+5002', '3FGL J1224.5+4957', '3FGL J1228.7+4857', '3FGL J1248.0+5130', '3FGL J1253.2+5300', '3FGL J1310.7+5515', '3FGL J1312.7+4828', '3FGL J1330.9+5201', '3FGL J1331.8+4718', '3FGL J1333.7+5057', '3FGL J1341.5+5517', '3FGL J1359.0+5544', '3FGL J1415.2+4832', '3FGL J1419.9+5425', '3FGL J1436.8+5639', '3FGL J1440.1+4955', '3FGL J1442.6+5156', '3FGL J1450.9+5200', '3FGL J1454.5+5124', '3FGL J1500.6+4750', '3FGL J1503.7+4759']
S_gamma       = [] # gamma-ray energy flux in erg/cm^2/s
S_gamma_error = [] # gamma-ray energy flux uncertainty in erg/cm^2/s
Gamma         = [] # gamma-ray photon spectral index
Gamma_error   = [] # gamma-ray photon spectral index uncertainty
S_r           = get_data(7, filename) # radio flux density in Jansky
S_r_error     = get_data(8, filename) # radio flux density uncertainty in Jansky
alpha         = get_data(9, filename) # spectral index
alpha_error   = get_data(10, filename) # spectral index uncertainty
z             = get_data(11, filename) # redshift
Delta_nu      = (168 - 140) * 1e6 # bandwidth in Hertz

h = fits.open('../../data/3fgl.fits') # downloaded from https://fermi.gsfc.nasa.gov/ssc/data/access/lat/4yr_catalog/gll_psc_v16.fit
Source_Name        = h[1].data['Source_Name'       ]
Energy_Flux100     = h[1].data['Energy_Flux100'    ]
Unc_Energy_Flux100 = h[1].data['Unc_Energy_Flux100']
Spectral_Index     = h[1].data['Spectral_Index'    ]
Unc_Spectral_Index = h[1].data['Unc_Spectral_Index']
h.close()

for source in sources:
    for i in range(len(Source_Name)):
        if source == Source_Name[i]:
            S_gamma.append(Energy_Flux100[i])
            S_gamma_error.append(Unc_Energy_Flux100[i])
            Gamma.append(Spectral_Index[i])
            Gamma_error.append(Unc_Spectral_Index[i])

# Calculate $G$

In [3]:
G, G_error, G_alpha_not_0 = [], [], []

for i in range(len(sources)):
    G.append((S_gamma[i] / (1e-23 * Delta_nu * S_r[i])) * (1 / ((1 + z[i]) ** (1 - Gamma[i]))))
    
ln_G, ln_G_error = log(G), []

for i in range(len(sources)):
    G_error.append(sqrt(
    ((1 / (1e-23 * Delta_nu * S_r_error[i])) * (1 / ((1 + z[i]) ** (1 - Gamma[i]))) * S_gamma_error[i]) ** 2 +
    ((- S_gamma[i] / (1e-23 * Delta_nu * (S_r[i] ** 2))) * (1 / ((1 + z[i]) ** (1 - Gamma[i]))) * S_r_error[i]) ** 2 +
    ((S_gamma[i] / (1e-23 * Delta_nu * S_r[i])) * (log(z[i] + 1) / ((1 + z[i]) ** (1 - Gamma[i]))) * Gamma_error[i]) ** 2))
    
for i in range(len(sources)):
    ln_G_error.append(G_error[i] / G[i])
    
for i in range(len(sources)):
    G_alpha_not_0.append((S_gamma[i] / (1e-23 * Delta_nu * S_r[i])) * (((1 + z[i]) ** (1 - alpha[i])) / ((1 + z[i]) ** (2 - Gamma[i]))))
    
ln_G_alpha_not_0 = log(G_alpha_not_0)

# $G$ correlations

In [5]:
print('r(Gamma, G)                 = ' + str(round(corrcoef(Gamma, G)[1, 0], 2)))
print('r(Gamma, ln(G))             = ' + str(round(corrcoef(Gamma, ln_G)[1, 0], 2)))
print('r(Gamma, ln(G_alpha_not_0)) = ' + str(round(corrcoef(Gamma, ln_G_alpha_not_0)[1, 0], 2)))

r(Gamma, G)                 = -0.11
r(Gamma, ln(G))             = 0.07
r(Gamma, ln(G_alpha_not_0)) = 0.06


# $G$ for each subclass

In [11]:
SED_class = ['HSP', 'HSP', 'HSP', 'LSP', 'LSP', 'LSP', 'HSP', 'ISP', '-', 'LSP', 'ISP', 'LSP', 'LSP', 'LSP', 'LSP', 'LSP', 'ISP', 'HSP', 'ISP', 'ISP', 'LSP', 'HSP', 'LSP', '-', 'ISP', 'ISP', 'LSP', 'LSP']
blazar_class = ['bll', 'bll', 'bll', 'bcu II', 'fsrq', 'fsrq', 'bll', 'bll', 'fsrq', 'fsrq', 'bll', 'bll', 'fsrq', 'agn', 'bcu II', 'fsrq', 'fsrq', 'bll', 'fsrq', 'bll', 'bll', 'bll', 'bll', 'rdg', 'bll', 'bll', 'bll', 'bll']

BL_Lac, BL_Lac_error, FSRQ, FSRQ_error, other, other_error = [], [], [], [], [], []
HSP, HSP_error, ISP, ISP_error, LSP, LSP_error = [], [], [], [], [], []

for i in range(len(sources)):
    if SED_class[i] == 'HSP':
        HSP.append(ln_G[i])
        HSP_error.append(ln_G_error[i])
    elif SED_class[i] == 'ISP':
        ISP.append(ln_G[i])
        ISP_error.append(ln_G_error[i])
    elif SED_class[i] == 'LSP':
        LSP.append(ln_G[i])
        LSP_error.append(ln_G_error[i])
        
    if blazar_class[i] == 'bll':
        BL_Lac.append(ln_G[i])
        BL_Lac_error.append(ln_G_error[i])
    elif blazar_class[i] == 'fsrq':
        FSRQ.append(ln_G[i])
        FSRQ_error.append(ln_G_error[i])
    else:
        other.append(ln_G[i])
        other_error.append(ln_G_error[i])

for i in range(len(BL_Lac)):
    BL_Lac_error[i] = BL_Lac_error[i] ** 2

for i in range(len(FSRQ)):
    FSRQ_error[i] = FSRQ_error[i] ** 2

for i in range(len(other)):
    other_error[i] = other_error[i] ** 2
    
for i in range(len(HSP)):
    HSP_error[i] = HSP_error[i] ** 2

for i in range(len(ISP)):
    ISP_error[i] = ISP_error[i] ** 2

for i in range(len(LSP)):
    LSP_error[i] = LSP_error[i] ** 2
    
print('<HSP>    = ', str(round(mean(HSP), 2)), '+/-', str(round(sqrt(sum(HSP_error)), 2)), '(N =', str(len(HSP)) + ')')
print('<ISP>    = ', str(round(mean(ISP), 2)), '+/-', str(round(sqrt(sum(ISP_error)), 2)), '(N =', str(len(ISP)) + ')')
print('<LSP>    = ', str(round(mean(LSP), 2)), '+/-', str(round(sqrt(sum(LSP_error)), 2)), '(N =', str(len(LSP)) + ')\n')        
        
print('<BL Lac> = ', str(round(mean(BL_Lac), 2)), '+/-', str(round(sqrt(sum(BL_Lac_error)), 2)), '(N =', str(len(BL_Lac)) + ')')
print('<FSRQ>   = ', str(round(mean(FSRQ), 2)), '+/-', str(round(sqrt(sum(FSRQ_error)), 2)), '(N =', str(len(FSRQ)) + ')')
print('<other>  = ', str(round(mean(other), 2)), '+/-', str(round(sqrt(sum(other_error)), 2)), '(N =', str(len(other)) + ')')

<HSP>    =  13.48 +/- 2.09 (N = 6)
<ISP>    =  13.43 +/- 1.91 (N = 7)
<LSP>    =  12.38 +/- 2.69 (N = 13)

<BL Lac> =  12.94 +/- 3.21 (N = 16)
<FSRQ>   =  12.55 +/- 1.84 (N = 8)
<other>  =  12.13 +/- 2.1 (N = 4)
